## Simulation Scenario

An earthquake has struck NovaCity, causing widespread destruction, including fires, and road blockages. Emergency response teams need to be coordinated for rescue operations.

The city is divided into **5 regions (R1 to R5)**. Each region has an evaluated **damage level** according to its blocked roads and fires:
- High damage (H): $>=50\%$ roads blocked, or $>=30\%$ fires;
- Medium damage (M): $[10\%, 50\%)$ roads blocked, or $[10\%, 30\%)$ fires; and
- Low damage (L): $<10\%$ roads blocked, or $<10\%$ fires.

Either blocked roads or fires that reach the range can be considered its damage level. For example, a region with 30\% road blocks and 5\% fires is Medium damaged.

The **initial damage** to each region is provided:
- R1 (H): 60\% roads blocked, 35\% fires;
- R2 (M): 40\% roads blocked, 25\% fires;
- R3 (M): 15\% roads blocked, 5\% fires;
- R4 (H): 35\% roads blocked, 30\% fires; and
- R5 (L): 5\% roads blocked, 3\% fires.

Assume all regions are of equal size to simplify the modelling process. Each region is modelled as a **node** in a graph, and predefined **distances** between regions (e.g., 5 km between R1 and R2, 3 km between R2 and R3) are detailed: R1 -- R2: 5 km, R1 -- R3: 7 km, R1 -- R4: 4 km, R1 -- R5: 6 km, R2 -- R3: 3 km, R2 -- R4: 4 km, R2 -- R5: 8 km, R3 -- R4: 5 km, R3 -- R5: 6 km, and R4 -- R5: 4 km.

During the rescue, each region will continue to experience fire spread and aftershocks. Fires spread every 10 minutes within the same region, increasing the fire percentage by 10\%. Aftershocks occur randomly every 15 minutes increasing road blockages by 10\% over all regions. For example, in R1, the initial damage is 60\% roads blocked and 35\% fires. After 10 minutes, it becomes $(60\%, 45\%)$, and after an additional 5 minutes, it becomes $(70\%, 45\%)$ if no rescue operations are performed.

Rescue agents are distributed across regions at the start of the simulation. **Eight units of fire trucks start at R2.** Each fire truck unit decreases fire percentage by 10\% per rescue operation. **Six units of police start at R4.** Each police unit decreases road blockages by 10\% per rescue operation. Multiple agents/units can perform rescue operations at the same time in the same region. The effects of their actions are cumulative. For example, if 2 fire trucks are deployed to R1 simultaneously, the fire percentage decreases by 20\% in one operation. Rescue operations affect the regional damage directly. If a fire truck reduces fire percentage by 10\%, that 10\% decrease is applied to the overall fire damage in that region, e.g., $20\%-10\%=10\%$.

**Assumption:** Both rescue operations and disaster events (fire spread and aftershocks) do not consume time in this simulation, but each unit can only perform rescue once when visiting the region. Rescue agents will not lose resources along a path, and all operations will occur instantly for the purpose of modelling. However, the effects of disasters and rescues will still be cumulative over time. Rescue agents aim to follow a path that covers all regions, ensuring each region is visited only once. After completing their assigned rescue tasks along the path, they must return to their starting point for a refill.

**Travel Speed:** Rescue agents can travel at a maximum speed of 60 km/h on unblocked roads. If roads are blocked, their speed will be reduced according to the percentage of blocked roads in the region they are traveling to or from. The travel speed between two regions is determined by the average percentage of blocked roads in both regions. For example, if R1 has 60\% blocked roads and R2 has 40\% blocked roads, the average blocked road percentage is $(60\% + 40\%) / 2 = 50\%$. The travel speed will then be reduced by 50\%, resulting in a travel speed of 30 km/h. Rescue agents won’t block each other. Fire damage will not affect travel speed.

Rescue operations must be completed within 90 minutes and the sooner the better. You need to allocate resources efficiently, prioritize critical regions, and adapt to changing conditions like fire spread and aftershocks. Rescue agents must quickly respond to new blockages and worsening fires, adjusting strategies to minimize delays.

# Optimizing Disaster Response Operations Using Multi-Method AI Techniques

## Overall Design

In this assignment, I explored an interconnected approach rather than a linear sequence for the objectives, as I found this to be the most effective way to address the complex requirements. The output from each task often feeds directly into the next, creating a cohesive system that adapts dynamically to evolving conditions.

For example, while working on Task 2 (Ant Colony Optimization), I generated optimized rescue paths based on real-time conditions. I then used these paths as the initial population for Task 1 (Genetic Algorithms for Path Optimization), allowing the GA to refine and build upon the ACO’s output. This integration strengthened my path optimization, enabling faster and more accurate resource allocation.

Similarly, I leveraged the output from the GA to guide decision-making in Task 3, where the Minimax algorithm with Alpha-Beta Pruning handles complex scenarios under competitive conditions. Tasks 4 and 5 (Game Theory and Bayesian Networks) further supported this design by incorporating strategic and probabilistic reasoning, enhancing the model’s ability to manage resource allocation and uncertainty.

By designing the solution as an interconnected system, each component builds on the others, resulting in a disaster response model that is both adaptive and highly responsive to the simulated environment.

## Pre-required functions and declarations

### Define Regions and Initial Damage Levels
This section defines the regions affected by the disaster, initializing each region with roads blocked, fires, and an initial damage level based on predefined thresholds.

In [ ]:
# Define regions with initial damage levels
regions = {
    "R1": {"roads_blocked": 60, "fires": 35, "damage_level": "H"},
    "R2": {"roads_blocked": 40, "fires": 25, "damage_level": "M"},
    "R3": {"roads_blocked": 15, "fires": 5, "damage_level": "M"},
    "R4": {"roads_blocked": 35, "fires": 30, "damage_level": "H"},
    "R5": {"roads_blocked": 5, "fires": 3, "damage_level": "L"}
}

### Define Distances Between Regions
The distances dictionary establishes the bidirectional distances (in km) between each region pair, enabling travel time calculations.

In [ ]:
# Define distances between regions (in km), bidirectional for easier access
distances = {
    ("R1", "R2"): 5, ("R2", "R1"): 5,
    ("R1", "R3"): 7, ("R3", "R1"): 7,
    ("R1", "R4"): 4, ("R4", "R1"): 4,
    ("R1", "R5"): 6, ("R5", "R1"): 6,
    ("R2", "R3"): 3, ("R3", "R2"): 3,
    ("R2", "R4"): 4, ("R4", "R2"): 4,
    ("R2", "R5"): 8, ("R5", "R2"): 8,
    ("R3", "R4"): 5, ("R4", "R3"): 5,
    ("R3", "R5"): 6, ("R5", "R3"): 6,
    ("R4", "R5"): 4, ("R5", "R4"): 4
}

### Initial Allocation of Resources
Agents (fire trucks and police units) are distributed initially, with eight fire trucks in R2 and six police units in R4.

In [ ]:
# Initial allocation of agents in regions
fire_truck_units = {"R2": 8}  # 8 fire truck units start in region R2
police_units = {"R4": 6}      # 6 police units start in region R4

# Define total available units for police and fire trucks
total_police_units = 6
total_fire_truck_units = 8

### Define Base Speed and Speed Adjustment
The function calculate_speed() calculates the travel speed between two regions based on their average road blockage percentage, adjusting from a base speed of 60 km/h.

In [ ]:
# Define base speed for travel on unblocked roads
base_speed = 60  # km/h

# Function to calculate adjusted speed based on average road blockages between regions
def calculate_speed(region1, region2):
    avg_blocked = (regions[region1]["roads_blocked"] + regions[region2]["roads_blocked"]) / 2
    adjusted_speed = base_speed * (1 - avg_blocked / 100)
    return adjusted_speed

### Fire Spread and Aftershock Functions
These functions simulate disaster events:

* spread_fire(): Increases fire levels by 10% every 10 minutes.
* cause_aftershock(): Increases road blockages by 10% every 15 minutes.

In [ ]:
# Update function to simulate fire spread every 10 minutes
def spread_fire():
    for region, damage in regions.items():
        damage["fires"] = min(100, damage["fires"] + 10)
    print("Fire spread updated:", {r: regions[r]["fires"] for r in regions})

# Update function to simulate aftershocks every 15 minutes
def cause_aftershock():
    for region, damage in regions.items():
        damage["roads_blocked"] = min(100, damage["roads_blocked"] + 10)
    print("Aftershock updated:", {r: regions[r]["roads_blocked"] for r in regions})

### Fire Truck and Police Operations
These functions allow cumulative reduction in fires and blockages per rescue operation:

* fire_truck_rescue(): Reduces fire levels by 10% per unit deployed.
* police_rescue(): Reduces road blockages by 10% per unit deployed.

In [ ]:
# Fire truck operation to reduce fire in a specific region, allowing multiple units to operate cumulatively
def fire_truck_rescue(region, units):
    reduction = units * 10
    regions[region]["fires"] = max(0, regions[region]["fires"] - reduction)
    print(f"Fire trucks reduced fire in {region} by {reduction}%:", regions[region]["fires"])

# Police operation to reduce road blockages in a specific region, allowing multiple units to operate cumulatively
def police_rescue(region, units):
    reduction = units * 10
    regions[region]["roads_blocked"] = max(0, regions[region]["roads_blocked"] - reduction)
    print(f"Police reduced road blockage in {region} by {reduction}%:", regions[region]["roads_blocked"])

### Update and Prioritize Regions by Damage Level
The update_damage_levels() function recalculates the damage level of each region based on current fire and road block conditions, while get_prioritized_regions() returns a list of regions weighted by damage severity, ensuring high-damage regions are prioritized.

In [ ]:
# Function to assess and update the damage level of each region based on fire and road blockage levels
def update_damage_levels():
    for region, damage in regions.items():
        if damage["roads_blocked"] >= 50 or damage["fires"] >= 30:
            damage["damage_level"] = "H"
        elif 10 <= damage["roads_blocked"] < 50 or 10 <= damage["fires"] < 30:
            damage["damage_level"] = "M"
        else:
            damage["damage_level"] = "L"
    print("Damage levels updated:", {r: regions[r]["damage_level"] for r in regions})

# Function to prioritize regions based on damage level
def get_prioritized_regions():
    high_damage = [region for region, data in regions.items() if data["damage_level"] == "H"]
    medium_damage = [region for region, data in regions.items() if data["damage_level"] == "M"]
    low_damage = [region for region, data in regions.items() if data["damage_level"] == "L"]

    # Triple weight High-damage regions to prioritize them even more
    return high_damage * 3 + medium_damage + low_damage

MIN_FIRE_UNITS_FOR_HIGH_DAMAGE = 2  # Minimum fire units reserved for high-damage regions
MIN_POLICE_UNITS_FOR_HIGH_DAMAGE = 2  # Minimum police units reserved for high-damage regions

## Task 2: Ant Colony Optimization (ACO) for Multi-Agent Coordination

Goal:
The goal of Task 2 is to optimize the paths taken by rescue agents (fire trucks and police units) using Ant Colony Optimization. ACO enables these agents to find the best routes to regions with varying damage levels based on real-time updates of road blockages and fires. This approach simulates pheromone trails left by each agent, reinforcing optimal paths over time and improving the coordination between agents.

Integration:

* Input: Region distance data (distances), road blockage and fire severity data (regions), and initial pheromone levels for each path.
* Output: Optimized paths with dynamically updated pheromone trails reflecting current rescue priorities.
* Next Step: The paths generated by ACO become the initial population for Task 1 (Genetic Algorithms for Path Optimization), allowing the Genetic Algorithm to refine paths based on further rescue efficiency criteria.
* Equations:
$$\tau_{ij} \leftarrow \text{Evaporation}(\tau_{ij})$$

$$\tau_{ij} \leftarrow \text{Rescue}(\tau_{ij})$$ where `Evaporation` can be fire spread or aftershock, `Rescue` can be from a unit of fire truck or a unit of police.

$$\eta_{ij}=1/d_{ij}$$

$$p^k_{ij}=\frac{[\tau_{ij}]^\alpha[\eta_{ij}]^\beta}{\sum_{l\in N_k(i)}[\tau_{il}]^\alpha[\eta_{il}]^\beta}$$

### 1. Initialize Pheromone Levels
This section sets initial pheromone levels for paths used by both fire trucks and police units, establishing a starting point for path desirability.

In [ ]:
import random

# Initialize pheromone levels for both fire trucks and police paths
pheromone_fire = {edge: 1.0 for edge in distances.keys()}  # Initial pheromone levels for fire paths
pheromone_police = {edge: 1.0 for edge in distances.keys()}  # Initial pheromone levels for police paths

### 2. Set ACO Parameters
Parameters define how strongly agents are influenced by pheromone trails versus the travel time, and an evaporation rate simulates pheromone decay over time.

In [ ]:
# ACO parameters
alpha = 1            # Importance of pheromone level in decision-making
beta = 2             # Importance of desirability (inverse of travel time)
evaporation_rate = 0.85  # Evaporation factor simulating decay of pheromones

### 3. Pheromone Evaporation Function
This function reduces pheromone levels over time, simulating the natural decay of pheromone trails. This decay allows agents to find alternative paths rather than always following old trails.

In [ ]:
# Function to evaporate pheromones over time to encourage exploration of new paths
def evaporate_pheromones():
    for edge in pheromone_fire:
        pheromone_fire[edge] *= evaporation_rate
    for edge in pheromone_police:
        pheromone_police[edge] *= evaporation_rate

### 4. Pheromone Deposition Function
After a rescue operation, agents deposit pheromones along their paths. The amount of pheromone deposited depends on the units used, reinforcing paths that agents have found useful.

In [ ]:
# Function to deposit pheromones along the paths taken by fire trucks or police
def deposit_pheromones(path, units, pheromone_type="fire"):
    reinforcement = units * 0.1  # Amount of reinforcement per unit
    pheromone = pheromone_fire if pheromone_type == "fire" else pheromone_police
    for i in range(len(path) - 1):
        region1, region2 = path[i], path[i + 1]
        pheromone[(region1, region2)] += reinforcement
        pheromone[(region2, region1)] += reinforcement  # Paths are bidirectional

### 5. Calculate Edge Travel Time
This function calculates the time needed to travel between two regions, considering road blockages. It adjusts speed based on blockage percentages, ensuring that agents consider path difficulty when choosing routes.

In [ ]:
def edge_travel_time(region1, region2):
    avg_blocked = (regions[region1]["roads_blocked"] + regions[region2]["roads_blocked"]) / 2
    adjusted_speed = max(base_speed * (1 - avg_blocked / 100), 10)  # Minimum speed threshold

    distance = distances[(region1, region2)]
    travel_time = (distance / adjusted_speed) * 60  # Convert hours to minutes

    print(f"Travel from {region1} to {region2}: Distance = {distance} km, "
          f"Avg Blockage = {avg_blocked}%, Adjusted Speed = {adjusted_speed:.2f} km/h, "
          f"Travel Time = {travel_time:.2f} minutes")

    return travel_time


### 6. Calculate Desirability of Paths
Desirability is calculated as the inverse of travel time, allowing agents to prioritize quicker paths.

In [ ]:
# Calculate desirability based on inverse travel time for each path
def calculate_desirability():
    desirability = {}
    for (region1, region2) in distances.keys():
        travel_time = edge_travel_time(region1, region2)
        desirability[(region1, region2)] = 1 / travel_time if travel_time > 0 else 1  # Higher desirability for shorter travel times
    return desirability

### 7. Choose Next Region Based on Pheromones and Desirability
This function selects the next region for an agent to visit based on pheromone levels and path desirability. The probabilities encourage agents to choose paths with higher pheromone levels and lower travel times.

In [ ]:
# Function to choose the next region based on pheromone levels and desirability
def choose_next_region(current_region, pheromone, desirability):
    total = sum((pheromone[(current_region, neighbor)] ** alpha) *
                (desirability[(current_region, neighbor)] ** beta)
                for neighbor in regions if (current_region, neighbor) in pheromone)

    probabilities = {}
    for neighbor in regions:
        if (current_region, neighbor) in pheromone:
            tau_eta = (pheromone[(current_region, neighbor)] ** alpha) * (desirability[(current_region, neighbor)] ** beta)
            probabilities[neighbor] = tau_eta / total

    # Select next region based on calculated probabilities
    next_region = random.choices(list(probabilities.keys()), weights=probabilities.values())[0]
    return next_region

### 8. Simulate Agent Path
This function simulates the entire path an agent takes, accumulating travel time and updating pheromones along the way.

In [ ]:
# Function to simulate path taken by an agent, updating pheromone levels as they move
def simulate_agent_path(start_region, units, pheromone_type="fire"):
    path = [start_region]
    current_region = start_region
    pheromone = pheromone_fire if pheromone_type == "fire" else pheromone_police
    desirability = calculate_desirability()
    total_travel_time = 0  # Accumulate total travel time

    while len(path) < len(regions):
        next_region = choose_next_region(current_region, pheromone, desirability)
        path.append(next_region)
        travel_time = edge_travel_time(current_region, next_region)
        total_travel_time += travel_time
        current_region = next_region

    # Deposit pheromones after path simulation
    deposit_pheromones(path, units, pheromone_type)
    print(f"Total travel time for path {path}: {total_travel_time:.2f} minutes")
    return path, total_travel_time

## Summary

In Task 2, the Ant Colony Optimization (ACO) algorithm was implemented to simulate pathfinding for rescue agents (fire trucks and police units) based on real-time conditions such as road blockages and damage levels. Key steps in this task included:

* Pheromone Initialization and Update: Initial pheromone levels were set for each path, and as agents moved, pheromones were deposited along successful paths. An evaporation function also reduced pheromone levels over time, encouraging agents to explore new routes.
* Travel Time and Desirability Calculation: Travel times were dynamically adjusted based on road blockages, and desirability scores (based on inverse travel time) helped agents prioritize quicker paths.
* Decision-Making with Pheromone Trails: Agents selected their next region based on a combination of pheromone levels and desirability, simulating the ACO approach where paths with high pheromone levels become more attractive.

Through these steps, ACO provided a framework for adaptive and coordinated movement, with agents leaving pheromone trails to reinforce effective paths while adjusting to current disaster conditions.

How This Helps the Next Step:
The paths generated by ACO serve as the initial population for the Genetic Algorithm (GA) in Task 1, ensuring that GA begins with a set of paths already optimized for current conditions. The pheromone trails also guide the GA's selection process, as paths with higher pheromone levels reflect successful rescue operations, enhancing the GA’s ability to find efficient and effective paths in the subsequent task.

## Task 1: Genetic Algorithms for Path Optimization

Goal:
The objective of Task 1 is to optimize the paths for rescue agents by employing a Genetic Algorithm (GA). This algorithm uses principles of natural selection—such as crossover, mutation, and fitness evaluation—to find efficient rescue paths that minimize travel time while maximizing pheromone levels.

Integration:

* Input: The GA takes in the initial population of paths generated using Ant Colony Optimization (Task 2), region pheromone levels, and region distances.
* Output: The optimized path, which will be used as the primary path for each agent in the simulation.
* Next Step: The optimized path produced by GA is used in subsequent simulation steps to execute resource allocation, minimize travel time, and respond to updated damage levels efficiently.


### 1. Initialize Parameters
The parameters define population size, number of generations, and mutation rate for the GA.

In [ ]:
import random

# Parameters for Genetic Algorithm
population_size = 10       # Size of the path population in each generation
num_generations = 50       # Number of generations to evolve the population
mutation_rate = 0.1        # Probability of mutation in each path

### 2. Generate Initial Population
This function generates a random initial population of paths based on ACO pheromone data, which serves as the starting point for the GA.

In [ ]:
# Generate initial population of paths based on ACO path probabilities
def generate_initial_population(start_region, pheromone, size=population_size):
    population = []
    regions_list = list(regions.keys())
    regions_list.remove(start_region)

    # Define default probabilities if not provided
    probabilities = {region: 1.0 for region in regions}  # Uniform probabilities if none are provided

    for _ in range(size):
        path = [start_region]
        remaining_regions = regions_list[:]

        while remaining_regions:
            # Select next region based on probabilities
            region_weights = [probabilities.get(region, 1.0) for region in remaining_regions]
            next_region = random.choices(remaining_regions, weights=region_weights)[0]
            path.append(next_region)
            remaining_regions.remove(next_region)

        path.append(start_region)  # Return to starting point
        population.append(path)
    return population


### 3. Define Fitness Function
The fitness function evaluates paths by balancing travel distance and pheromone levels. A higher pheromone-to-distance ratio indicates a more efficient path.

In [ ]:
# Fitness function to evaluate paths based on total travel time and pheromone level
def fitness_function(path, pheromone):
    total_distance = sum(distances[(path[i], path[i + 1])] for i in range(len(path) - 1))
    pheromone_score = sum(pheromone[(path[i], path[i + 1])] for i in range(len(path) - 1))
    return (pheromone_score / total_distance)  # Higher pheromone and lower distance yield better fitness

### 4. Selection Function
This function selects parents for crossover based on fitness scores, favoring paths with higher fitness.

In [ ]:
# Selection function to pick parents based on fitness scores
def selection(population, pheromone):
    fitness_scores = [fitness_function(path, pheromone) for path in population]
    total_fitness = sum(fitness_scores)
    probabilities = [fitness / total_fitness for fitness in fitness_scores]
    parents = random.choices(population, weights=probabilities, k=2)
    return parents

### 5. Crossover Function
This function creates offspring by combining segments from two parent paths. The crossover operation introduces diversity in the population by merging parts of different paths.

In [ ]:
# Crossover function to create offspring by combining parts of two parents
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 2)
    child = parent1[:crossover_point] + [region for region in parent2 if region not in parent1[:crossover_point]]
    child.append(parent1[0])  # Ensure path returns to the starting region
    return child

### 6. Exchange (Mutation) Function
The exchange function randomly swaps regions in a path to introduce variations, helping avoid local optima.

In [ ]:
# Exchange function to swap regions in a path to introduce diversity
def exchange(path):
    i, j = random.sample(range(1, len(path) - 1), 2)
    path[i], path[j] = path[j], path[i]
    return path

### 7. Successor Function
This function applies mutation based on a predefined rate, allowing slight modifications to paths, enhancing exploration.

In [ ]:
# Successor function to generate the next best path by slight modification
def successor(path, pheromone):
    new_path = path[:]
    if random.random() < mutation_rate:
        new_path = exchange(new_path)
    return new_path

### 8. Main Genetic Algorithm Function
This function iteratively evolves the population over multiple generations, selecting the best paths for each generation. The best path is then returned as the optimized solution.

In [ ]:
# Main Genetic Algorithm function
def genetic_algorithm(start_region, probabilities, pheromone):
    # Generate initial population
    population = generate_initial_population(start_region, probabilities)

    for generation in range(num_generations):
        new_population = []

        # Generate new population using crossover and mutation
        while len(new_population) < population_size:
            parent1, parent2 = selection(population, pheromone)
            child = crossover(parent1, parent2)
            child = successor(child, pheromone)
            new_population.append(child)

        # Select the best paths to carry over to the next generation
        population = sorted(new_population, key=lambda path: fitness_function(path, pheromone), reverse=True)[:population_size]

    # Return the best path from the final generation
    best_path = max(population, key=lambda path: fitness_function(path, pheromone))
    return best_path, fitness_function(best_path, pheromone)

## Summary
In Task 1, the Genetic Algorithm (GA) was implemented to optimize rescue paths for agents based on travel time and pheromone levels. This task involved several key components:

* Initial Population Generation: Using ACO-generated pheromone data, a diverse set of initial paths was created, ensuring that the GA starts with viable routes.
* Fitness Evaluation: Each path was evaluated with a fitness function that considers both travel distance and pheromone intensity, encouraging paths that balance efficiency and existing trail strength.
* Selection, Crossover, and Mutation: The GA used natural selection principles, with selection favoring high-fitness paths, crossover creating new paths by combining parent paths, and mutation adding variation to avoid local optima.

By iterating through these steps across generations, the GA refined the population of paths, eventually producing an optimized path with minimal travel time and strong pheromone levels.

How This Helps the Next Step:  
The output from the GA—the optimized rescue path—is a critical input for the disaster response simulation. This path provides a well-coordinated route for agents, enabling them to efficiently cover all regions and prioritize high-damage areas. This optimized path also informs subsequent rescue operations, ensuring that resource allocation and response time are minimized as the simulation moves forward.

## Task 3: Minmax Algorithm with Alpha-Beta Pruning and Enhancements

Goal:  
The goal of Task 3 is to optimize resource allocation in competitive rescue scenarios. Using the Minimax algorithm with Alpha-Beta pruning, this approach models the interaction between rescue agents (maximizing player) and disaster effects (minimizing player). The algorithm aims to maximize resource effectiveness by assigning optimal units to regions while considering disaster spread as a competing factor.

Integration:  
* Input: Optimized paths generated from the Genetic Algorithm (Task 1) and ACO-derived region priorities. The initial resources (fire trucks and police units) and current region conditions (road blockages and fire levels) are also inputs.
* Output: Optimal allocation of fire and police units for each region along the prioritized path, ensuring maximum effectiveness against the spread of disaster conditions.
* Next Step: The optimized resource allocation informs the main simulation, enabling agents to respond efficiently to evolving conditions in real-time.

Who is Max Player?  
The Max Player is the rescue team (fire trucks and police units). The Max Player’s goal is to maximize the utility by effectively allocating resources to reduce fire levels and clear road blockages in each region. In each turn, the rescue team aims to achieve the highest possible impact through optimal deployment of resources.

Who is Min Player?  
The Min Player represents the disaster effects, such as fire spread and road blockages. The Min Player’s objective is to minimize the effectiveness of the rescue efforts by simulating adverse conditions. This includes increasing fire levels and road blockages, essentially adding obstacles and testing the resilience of the rescue team's strategy.

Game Tree Construction:  
Any data should be prepared by pre-computation? Utility?

Pre-computation of Data:  
To efficiently construct the game tree, certain data, such as region damage levels and prioritized paths, should be prepared in advance. Pre-computing the maximum allowed units for each region based on its damage level (e.g., more resources for high-damage regions) can streamline the Minimax evaluations.
Utility Function: The utility function should be defined based on the impact of resource allocation in reducing fire and road blockages. Utility can be pre-computed or cached for specific allocations in regions with similar conditions to reduce computational overhead.

### 1. Define Minimax Function with Alpha-Beta Pruning
This function uses the Minimax algorithm with Alpha-Beta pruning to optimize resource allocation. The function explores possible actions by rescue agents, simulating resource allocation for both the maximizing player (rescue) and minimizing player (disaster spread).

In [ ]:
def minimax(path, depth, remaining_fire_units, remaining_police_units, alpha, beta, maximizing_player):
    if depth == len(path):  # Base case: end of path
        return 0  # Utility at the terminal node

    current_region = path[depth]
    max_units_for_region = MAX_UNITS_PER_REGION if regions[current_region]["damage_level"] == "H" else 2  # Cap for Medium regions

    # Maximizing player's turn (Rescue team)
    if maximizing_player:
        max_utility = float('-inf')
        for fire_allocation in range(min(remaining_fire_units, max_units_for_region) + 1):
            for police_allocation in range(min(remaining_police_units, max_units_for_region) + 1):
                utility = evaluate_utility(current_region, fire_allocation, police_allocation)
                remaining_fire = remaining_fire_units - fire_allocation
                remaining_police = remaining_police_units - police_allocation
                utility += minimax(path, depth + 1, remaining_fire, remaining_police, alpha, beta, False)
                max_utility = max(max_utility, utility)
                alpha = max(alpha, utility)
                if beta <= alpha:  # Alpha-Beta Pruning
                    break
            if beta <= alpha:
                break
        return max_utility

    # Minimizing player's turn (Disaster spread)
    else:
        min_utility = float('inf')
        for fire_spread in range(10, 30 + 1, 10):  # Example fire spread increments
            for blockage_increase in range(10, 30 + 1, 10):  # Example blockage increments
                # Calculate the impact of disaster conditions
                regions[current_region]["fires"] = min(100, regions[current_region]["fires"] + fire_spread)
                regions[current_region]["roads_blocked"] = min(100, regions[current_region]["roads_blocked"] + blockage_increase)

                # Recursively call minimax for the next region, maximizing player's turn
                utility = -evaluate_utility(current_region, remaining_fire_units, remaining_police_units)
                utility += minimax(path, depth + 1, remaining_fire_units, remaining_police_units, alpha, beta, True)

                # Reset region damage after simulation
                regions[current_region]["fires"] -= fire_spread
                regions[current_region]["roads_blocked"] -= blockage_increase

                # Update min utility and beta for minimizing player
                min_utility = min(min_utility, utility)
                beta = min(beta, utility)

                if beta <= alpha:  # Alpha-Beta Pruning
                    break
            if beta <= alpha:
                break
        return min_utility

### 2. Optimal Resource Allocation Function
This function uses the Minimax algorithm to determine the optimal allocation of fire and police units for each region along a prioritized path. It iterates through feasible allocations, maximizing utility by using the Minimax decision tree.

In [ ]:
def optimal_resource_allocation(path, initial_fire_units, initial_police_units):
    prioritized_path = get_prioritized_regions()
    best_utility = float('-inf')
    best_fire_allocation, best_police_allocation = 0, 0

    # Limit the allocation to what is available
    max_fire_allocation = min(initial_fire_units, remaining_fire_units)
    max_police_allocation = min(initial_police_units, remaining_police_units)

    for fire_allocation in range(1, max_fire_allocation + 1):
        for police_allocation in range(1, max_police_allocation + 1):
            utility = minimax(prioritized_path, 0, fire_allocation, police_allocation, float('-inf'), float('inf'), True)
            if utility > best_utility:
                best_utility, best_fire_allocation, best_police_allocation = utility, fire_allocation, police_allocation

    return best_fire_allocation, best_police_allocation

## Summary
In Task 3, the Minimax algorithm with Alpha-Beta pruning was implemented to optimize the allocation of fire trucks and police units for each region. This process involved:

* Maximizing Resource Utility:  
By alternating turns between the rescue team (maximizing player) and the disaster effects (minimizing player), the Minimax function determines the most effective allocation of resources.
* Alpha-Beta Pruning:  
Alpha-Beta pruning reduces unnecessary computations by ignoring suboptimal branches, making the decision-making process more efficient.
* Simulating Disaster Spread:  
By incorporating the disaster’s impact on the regions (such as fire spread and road blockage increases), the algorithm considers worst-case scenarios, allowing for preemptive planning.

How This Helps the Next Step:  
The output from Task 3—optimized allocations of fire trucks and police units—is used directly in the main simulation. These allocations ensure that resources are deployed where they are most needed, balancing response time and the scale of regional damage. This step supports the real-time responsiveness of the overall disaster response system, equipping agents with a strategic plan for dealing with worsening conditions efficiently.

## Task 4: Game Theory for Multi-Agent Systems

Goal:  
The goal of Task 4 is to use game theory to determine the most effective allocation strategy for fire and police units based on their combined damage reduction impact. By modeling different allocation combinations in a payoff matrix, this task enables a strategic analysis of unit allocation, maximizing damage reduction across regions. This approach supports identifying optimal strategies for resource deployment, considering the interactions between police and fire unit allocations.

Integration:  

* Input: Available units for police and fire trucks, region states (road blockages and fire levels), and possible allocation options.
* Output: A payoff matrix that represents the effectiveness of different allocation combinations for damage reduction.
* Next Step: The payoff matrix from Task 4 helps to identify equilibrium or dominant strategies, guiding the actual allocation of resources in the disaster response simulation.

### 1. Create Payoff Matrix Function
This function creates an empty payoff matrix based on the possible allocation options for police and fire units. Each cell will represent the total damage reduction achieved by a specific allocation combination.

In [ ]:
def create_payoff_matrix(police_allocation_options, fire_allocation_options):
    """Creates and initializes an empty payoff matrix based on possible allocations."""
    return [[0 for _ in range(len(police_allocation_options))] for _ in range(len(fire_allocation_options))]

### 2. Evaluate Strategy Function
This function iterates over each allocation combination for fire and police units, calculating the total damage reduction. The matrix is populated based on the combined impact of police and fire allocations, reflecting how effectively each strategy combination reduces regional damage.

In [ ]:
# Enhanced payoff matrix for prioritizing high-impact allocations
def evaluate_optimized_strategy(matrix, police_actions, fire_actions, regions_state):
    for i, f_units in enumerate(fire_actions):
        for j, p_units in enumerate(police_actions):
            damage_reduction = 0
            for region, damage in regions_state.items():
                police_reduction = min(damage["roads_blocked"], p_units * 10)
                fire_reduction = min(damage["fires"], f_units * 10)
                damage_reduction += police_reduction + fire_reduction
            # Weigh higher impact actions more heavily
            matrix[i][j] = damage_reduction * (1.1 if damage_reduction > 50 else 1.0)
    return matrix

### 3. Define Police and Fire Allocation Options
These lists define the allocation options available for both police and fire units based on their respective maximum units. Each option represents a possible number of units that can be deployed, and these are used to populate the payoff matrix.

In [ ]:
# Define police and fire unit allocation options based on available units
police_actions = list(range(1, total_police_units + 1))  # Options from 1 to max police units
fire_actions = list(range(1, total_fire_truck_units + 1))  # Options from 1 to max fire trucks

### 4. Create and Evaluate Payoff Matrix
The payoff matrix is created and then evaluated using the evaluate_strategy function. The matrix is printed to display the effectiveness of different allocation combinations.

In [ ]:
# Create payoff matrix and evaluate strategies
payoff_matrix = create_payoff_matrix(police_actions, fire_actions)
payoff_matrix = evaluate_strategy(payoff_matrix, police_actions, fire_actions, regions)

# Display the payoff matrix
print("Payoff Matrix (Damage Reduction):")
for row in payoff_matrix:
    print(row)

Payoff Matrix (Damage Reduction):
[83, 118, 148, 173, 183, 193]
[113, 148, 178, 203, 213, 223]
[138, 173, 203, 228, 238, 248]
[143, 178, 208, 233, 243, 253]
[143, 178, 208, 233, 243, 253]
[143, 178, 208, 233, 243, 253]
[143, 178, 208, 233, 243, 253]
[143, 178, 208, 233, 243, 253]


## Summary
In Task 4, game theory principles were applied to analyze resource allocation strategies, focusing on maximizing damage reduction through effective police and fire unit allocations. This process involved:

* Constructing a Payoff Matrix: The payoff matrix mapped the effectiveness of different allocation combinations for police and fire units, representing the impact on damage reduction.
* Evaluating Strategies: Each cell in the matrix was populated based on calculated damage reduction for each allocation pair, helping identify strategies with high effectiveness.  

How This Helps the Next Step:  
The payoff matrix from Task 4 serves as a foundation for determining equilibrium or dominant strategies in resource allocation. By understanding the impact of different allocations, the simulation can prioritize unit deployment that maximizes overall damage reduction, informing strategic decisions in real-time response efforts.

## Task 5: Bayesian Networks for Uncertain Inferences

Goal:  
The goal of Task 5 is to use a Bayesian Network to model the uncertainties in rescue operations. By defining relationships between factors like fire severity, road blockages, travel time, and rescue effectiveness, the network provides probabilistic insights into how these variables impact each other. This probabilistic model supports making informed decisions under uncertainty, allowing agents to adapt their actions based on the likely outcomes of different scenarios.

Integration:  

* Input:  
The Bayesian Network uses prior probabilities for fire severity, road blockages, and resource availability, along with conditional dependencies that affect travel time and rescue effectiveness.
* Output:  
The network produces inference results that quantify the likelihood of high, medium, or low rescue effectiveness under specific conditions, such as travel time and resource availability.
* Next Step:  
The inference results from the Bayesian Network guide the allocation and prioritization of resources, allowing the simulation to make probabilistically informed decisions based on real-time data and evolving disaster conditions.


### 1. Define the Bayesian Network Structure
This section establishes the structure of the Bayesian Network by specifying dependencies between variables. Here, FS (Fire Severity) and RB (Road Blockages) influence TT (Travel Time), while TT and RA (Resource Availability) impact RE (Rescue Effectiveness).

In [ ]:
#Install pgmpy
!pip install pgmpy
from pgmpy.models import BayesianNetwork

# Step 1: Define the structure of the Bayesian Network
bayesian_model = BayesianNetwork([
    ("FS", "TT"),  # Fire Severity affects Travel Time
    ("RB", "TT"),  # Road Blockages affect Travel Time
    ("TT", "RE"),  # Travel Time affects Rescue Effectiveness
    ("RA", "RE")   # Resource Availability affects Rescue Effectiveness
])

### 2. Define Conditional Probability Tables (CPTs)
Each node in the Bayesian Network is assigned a Conditional Probability Table (CPT), specifying the probabilities for different states. For instance, FS (Fire Severity) has three states with defined prior probabilities, while TT (Travel Time) depends on both FS and RB.

In [ ]:
from pgmpy.factors.discrete import TabularCPD

# Fire Severity (FS) - Prior probabilities
cpd_fs = TabularCPD(variable="FS", variable_card=3, values=[[0.3], [0.5], [0.2]],
                    state_names={"FS": ["Low", "Medium", "High"]})

# Road Blockages (RB) - Prior probabilities
cpd_rb = TabularCPD(variable="RB", variable_card=3, values=[[0.4], [0.4], [0.2]],
                    state_names={"RB": ["Low", "Medium", "High"]})

# Travel Time (TT) - Conditional on FS and RB
cpd_tt = TabularCPD(variable="TT", variable_card=3,
                    values=[
                        [0.8, 0.6, 0.4, 0.7, 0.5, 0.3, 0.6, 0.3, 0.1],  # Low Travel Time
                        [0.15, 0.3, 0.4, 0.2, 0.4, 0.5, 0.3, 0.5, 0.4], # Medium Travel Time
                        [0.05, 0.1, 0.2, 0.1, 0.1, 0.2, 0.1, 0.2, 0.5]  # High Travel Time
                    ],
                    evidence=["FS", "RB"], evidence_card=[3, 3],
                    state_names={"TT": ["Low", "Medium", "High"], "FS": ["Low", "Medium", "High"], "RB": ["Low", "Medium", "High"]})

# Resource Availability (RA) - Prior probabilities
cpd_ra = TabularCPD(variable="RA", variable_card=3, values=[[0.5], [0.3], [0.2]],
                    state_names={"RA": ["Low", "Medium", "High"]})

# Rescue Effectiveness (RE) - Conditional on TT and RA
cpd_re = TabularCPD(variable="RE", variable_card=3,
                    values=[
                        [0.7, 0.6, 0.4, 0.6, 0.5, 0.3, 0.4, 0.3, 0.2],  # High Effectiveness
                        [0.2, 0.3, 0.4, 0.3, 0.4, 0.5, 0.4, 0.5, 0.4],  # Medium Effectiveness
                        [0.1, 0.1, 0.2, 0.1, 0.1, 0.2, 0.2, 0.2, 0.4]   # Low Effectiveness
                    ],
                    evidence=["TT", "RA"], evidence_card=[3, 3],
                    state_names={"RE": ["High", "Medium", "Low"], "TT": ["Low", "Medium", "High"], "RA": ["Low", "Medium", "High"]})


### 3. Add CPTs to the Bayesian Network
This step integrates the Conditional Probability Tables (CPTs) into the Bayesian model, defining the behavior of each variable based on its dependencies.

In [ ]:
# Step 3: Add the CPDs to the model
bayesian_model.add_cpds(cpd_fs, cpd_rb, cpd_tt, cpd_ra, cpd_re)

### 4. Validate the Model Structure
The check_model function validates the network structure to ensure that all dependencies are correctly established and that the model is mathematically sound.

In [ ]:
# Step 4: Validate the model structure
assert bayesian_model.check_model()

### 5. Perform Inference
Using Variable Elimination, we can query the network to make probabilistic predictions. In this example, we infer the probability of achieving high rescue effectiveness given specific conditions for travel time and resource availability.

In [ ]:
from pgmpy.inference import VariableElimination

# Step 5: Perform inference to optimize rescue decisions
inference = VariableElimination(bayesian_model)

# Example inference query: What is the probability of High Rescue Effectiveness given Medium Travel Time and High Resource Availability?
query_result = inference.query(variables=["RE"], evidence={"TT": "Medium", "RA": "High"})
print("Inference Result (Probability of High Rescue Effectiveness):")
print(query_result)

Inference Result (Probability of High Rescue Effectiveness):
+------------+-----------+
| RE         |   phi(RE) |
+============+===========+
| RE(High)   |    0.3000 |
+------------+-----------+
| RE(Medium) |    0.5000 |
+------------+-----------+
| RE(Low)    |    0.2000 |
+------------+-----------+


## Summary
In Task 5, a Bayesian Network was developed to model uncertainty in rescue operations, incorporating factors like fire severity, road blockages, travel time, and resource availability. Key steps included:

* Defining the Network Structure and Dependencies:  
Relationships between key variables were established, allowing each factor to influence related outcomes.
* Adding Conditional Probability Tables:  
Probabilities were assigned based on expected behaviors, enabling the network to compute how conditions impact rescue effectiveness.
* Performing Inference:  
The model allows probabilistic queries that provide insights into the likelihood of achieving specific levels of rescue effectiveness under given conditions.

How This Helps the Next Step:
The Bayesian Network provides critical probabilistic insights for real-time decision-making. By evaluating the probability of successful rescue efforts under different conditions, the simulation can adjust strategies, prioritize regions, and allocate resources with a data-driven approach. This support for uncertainty handling ensures that rescue operations are better adapted to evolving conditions and resource availability, enhancing overall disaster response effectiveness.

# Final Simulation

The final simulation orchestrates all the previously defined components, such as resource allocation (Genetic Algorithm, Task 1), adaptive pathfinding (Ant Colony Optimization, Task 2), strategic deployment (Minimax with Alpha-Beta Pruning, Task 3), resource strategy evaluation (Game Theory Payoff Matrix, Task 4), and uncertainty handling (Bayesian Network, Task 5). This integrated simulation follows a path optimized by the GA, performing resource allocation and adapting to disaster spread events in real time.

Below is the code breakdown, incorporating functions from each task and running the final simulation loop to produce an overall summary of the disaster response results.

## 1. Simulation Setup and Code Breakdown
Simulation Parameters and Initialization
Here we define the maximum simulation time (MAX_TIME) and set up the initial path and resources based on the GA results.

In [ ]:
import time

# Simulation parameters
MAX_TIME = 90  # Maximum simulation time in minutes
TIME_STEP = 10  # Time step in minutes

# Initialize starting path from the GA (assuming path has already been calculated)
start_region = "R2"  # Starting point based on initial fire truck allocation
best_path, _ = genetic_algorithm(start_region, pheromone_fire, pheromone_fire)

# Set initial allocations for remaining units based on starting regions
remaining_fire_units = fire_truck_units.get("R2", 0)
remaining_police_units = police_units.get("R4", 0)

## 2. Simulate Disaster Events
This function triggers fire spread and aftershocks at specific intervals, as well as updating damage levels based on current conditions.

In [ ]:
# Function to simulate time step events (fire spread and aftershocks)
def simulate_disasters(elapsed_time):
    if elapsed_time % 10 == 0:  # Fire spread every 10 minutes
        spread_fire()
    if elapsed_time % 15 == 0:  # Aftershock every 15 minutes
        cause_aftershock()
    update_damage_levels()  # Update damage levels based on current conditions

Path evaluation

In [ ]:
# Function to dynamically re-evaluate the rescue path based on updated damage conditions
def reevaluate_path_due_to_disaster(best_path, fire_units, police_units):
    high_priority_regions = get_prioritized_regions()  # Get regions by damage priority
    new_path = [region for region in best_path if region in high_priority_regions]  # Prioritize high-damage regions
    if len(new_path) < len(best_path):  # Complete path with remaining regions
        new_path += [region for region in best_path if region not in new_path]
    return new_path

## 3. Rescue Operations Function
This function performs rescue operations by deploying resources until the fires and road blockages in a region are resolved.

In [ ]:
# Function to perform rescue operations and use full resources until resolution
def perform_rescue_operations(region):
    global remaining_fire_units, remaining_police_units
    initial_fire_units = remaining_fire_units
    initial_police_units = remaining_police_units

    # Deploy units until fires and blockages in the region are resolved
    while regions[region]["fires"] > 0 and remaining_fire_units > 0:
        fire_truck_rescue(region, 1)
        remaining_fire_units -= 1
    while regions[region]["roads_blocked"] > 0 and remaining_police_units > 0:
        police_rescue(region, 1)
        remaining_police_units -= 1

    print(f"\nRegion {region} - Rescue Operations:")
    print(f"  Fire units deployed: {initial_fire_units - remaining_fire_units}")
    print(f"  Police units deployed: {initial_police_units - remaining_police_units}")
    print(f"  Remaining Fire Units: {remaining_fire_units}, Remaining Police Units: {remaining_police_units}")
    print(f"  Updated Fires: {regions[region]['fires']}%, Roads Blocked: {regions[region]['roads_blocked']}%")

## 4. Main Simulation Loop
This loop iterates through each region in the optimized path, calculating travel time, performing rescue operations, and simulating disaster events. The loop stops if the maximum time is reached, and final region states are displayed.

In [ ]:
# Main simulation loop
elapsed_time = 0
print(f"\n--- Starting Simulation for {MAX_TIME} Minutes ---")

for i in range(len(best_path) - 1):
    current_region = best_path[i]
    next_region = best_path[i + 1]

    # Calculate travel time between regions considering road blockages
    travel_time = edge_travel_time(current_region, next_region)
    elapsed_time += travel_time

    # If we've exceeded the time limit, terminate the simulation
    if elapsed_time >= MAX_TIME:
        print("\nTime exceeded 90 minutes. Ending simulation.")
        break

    # Perform rescue operations in the next region
    perform_rescue_operations(next_region)

    # Disaster events simulation
    simulate_disasters(elapsed_time)
    elapsed_time += TIME_STEP  # Increment by TIME_STEP after every disaster event

    # Output travel time to next region
    print(f"Travel to {next_region} took {travel_time:.2f} minutes. Total elapsed time: {elapsed_time:.2f} minutes.")


--- Starting Simulation for 90 Minutes ---
Travel from R2 to R3: Distance = 3 km, Avg Blockage = 27.5%, Adjusted Speed = 43.50 km/h, Travel Time = 4.14 minutes
Fire trucks reduced fire in R3 by 10%: 0
Police reduced road blockage in R3 by 10%: 5
Police reduced road blockage in R3 by 10%: 0

Region R3 - Rescue Operations:
  Fire units deployed: 1
  Police units deployed: 2
  Remaining Fire Units: 7, Remaining Police Units: 4
  Updated Fires: 0%, Roads Blocked: 0%
Damage levels updated: {'R1': 'H', 'R2': 'M', 'R3': 'L', 'R4': 'H', 'R5': 'L'}
Travel to R3 took 4.14 minutes. Total elapsed time: 14.14 minutes.
Travel from R3 to R4: Distance = 5 km, Avg Blockage = 17.5%, Adjusted Speed = 49.50 km/h, Travel Time = 6.06 minutes
Fire trucks reduced fire in R4 by 10%: 20
Fire trucks reduced fire in R4 by 10%: 10
Fire trucks reduced fire in R4 by 10%: 0
Police reduced road blockage in R4 by 10%: 25
Police reduced road blockage in R4 by 10%: 15
Police reduced road blockage in R4 by 10%: 5
Police 

## 5. Display Final State
Once the simulation ends, the final state of each region (fires, road blockages, and damage level) is printed to summarize the disaster response results.

In [ ]:
# Final state output
print("\n--- Final State of All Regions ---")
for region, damage in regions.items():
    print(f"{region} - Fires: {damage['fires']}%, Roads Blocked: {damage['roads_blocked']}%, Damage Level: {damage['damage_level']}")

print("\n--- Simulation Complete ---")



--- Final State of All Regions ---
R1 - Fires: 0%, Roads Blocked: 60%, Damage Level: H
R2 - Fires: 25%, Roads Blocked: 40%, Damage Level: M
R3 - Fires: 0%, Roads Blocked: 0%, Damage Level: L
R4 - Fires: 0%, Roads Blocked: 0%, Damage Level: L
R5 - Fires: 3%, Roads Blocked: 5%, Damage Level: L

--- Simulation Complete ---


# Conclusion and Discussion

## Conclusion
The simulation of the disaster response operation provided valuable insights into resource allocation and decision-making during emergency situations. Over the course of the simulation, I learned the importance of adaptive strategies in response to dynamic conditions, as well as the interplay between fire spread, road blockages, and the efficacy of rescue operations.

### Learnings:

Resource Deployment: Effective management of fire trucks and police units is crucial. The simulation highlighted that while deploying resources, the order of operations significantly influences the outcome. High-priority regions must be addressed first to minimize overall damage and casualties.
Path Optimization: The integration of Genetic Algorithms and Ant Colony Optimization was instrumental in finding efficient paths for rescue operations. This reinforced the value of optimization techniques in real-time decision-making scenarios.
Impact of Disaster Dynamics: Understanding how quickly fires spread and road blockages develop allowed for more strategic planning of rescue missions. The iterative nature of the simulation demonstrated how quickly conditions could change, requiring ongoing assessment and adjustment of strategies.

### Improvements:

Real-time Feedback Mechanism: Implementing a real-time feedback mechanism to adjust resource allocation dynamically based on current conditions could enhance the simulation’s responsiveness. This would allow for immediate reallocation of resources as the situation evolves, rather than following a predetermined path.
Broader Unit Deployment: There were instances where not all available units were deployed. Incorporating a more strategic allocation algorithm that considers the number of remaining units and their optimal distribution across regions could prevent under-utilization of resources.
Comprehensive Reporting: While the simulation provided insights into the number of deployed units, a more comprehensive report detailing the impact of each unit’s deployment on fire reduction and road blockage clearance would enhance understanding of the effectiveness of the rescue operations.

### What Went Wrong:

The simulation did not effectively clear all fires and road blockages by the end of the run, indicating a possible misalignment in the deployment strategy or a lack of sufficient units allocated to high-impact regions. Certain regions continued to have significant damage levels, which suggests that rescue operations need to be better prioritized based on the severity of damage and the potential for resource allocation.
Some regions did not receive the expected number of fire and police units, which may indicate an issue in the logic for tracking and deploying units. This needs to be addressed to ensure that the simulation can accurately reflect the deployment of all available resources.
In summary, the simulation has proven to be a valuable learning tool, demonstrating the complexities of disaster response and the critical need for adaptable, data-driven strategies in emergency management. Future iterations should aim to refine the allocation algorithms and improve the responsiveness of the system to ensure that all regions can be effectively cleared of hazards within the simulation timeframe.